# Linear Models: OLS

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection, linear_model, neighbors, preprocessing, metrics 
from sklearn.preprocessing import Normalizer

In [56]:
clean = pd.read_csv("../data/clean.csv")
clean.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,NaN,millet,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,NaN,cowpea,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,NaN,NaN,257.234727,257.234727,0,NaN,NaN,africa
4,NaN,sesame,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa


In [57]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [58]:
clean.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,NaN,millet,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,NaN,cowpea,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,NaN,NaN,257.234727,257.234727,0,NaN,NaN,africa
4,NaN,sesame,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa


In [59]:
def standarize_data(df):
        """
        Input: a dataset
        action: returns numeric column values scaled by mean and standard deviation
        """
        numeric_data = df.select_dtypes(include=['float64', 'int64'])
        for i in numeric_data.columns:
            df[i] = (df[i] - df[i].mean())/df[i].std()
        return df

In [60]:
#for market orientation
#Clean up market orientation data
market_data = fun.delete_id_columns(clean) #1
print("1")
market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation", "PPI_Likelihood") #2
print("2")
market_data = fun.replace_NAN_with_na(market_data) #3
print("3")
market_data = fun.entry_to_lowercase(market_data) #4
print("4")
market_data = fun.remove_underscores_spaces(market_data) #5
print("5")
market_data = fun.convert_to_categorical(market_data) #6
print("6")
market_data = fun.impute_data(market_data)
print("7")
market_data = standarize_data(market_data)
market_data.describe()
market_data = market_data.drop("continent", axis = 1)
market_data.YEAR = market_data.YEAR.astype('category')

1
2
3
4
5
6
7


In [61]:
#code by ana
def get_dummyXs_y(df, y_var):
    
    y = df[y_var]
    X  = df.drop(y_var, axis = 1)
    X_cat = X.select_dtypes(include = ["category", "O"])
    X_num = X.select_dtypes(include=['float64', 'int64'])
    
    X_cat_dummy = pd.get_dummies(X_cat)
    newX = pd.concat([X_num, X_cat_dummy], axis = 1)
    
    return newX, y

In [62]:
market_nums = market_data.select_dtypes(include='number')
X_market = get_dummyXs_y(market_data, "Market_Orientation")[0]
y_market = get_dummyXs_y(market_data, "Market_Orientation")[1]

In [63]:
##Including non-numerics, but not as dummies
#X_plain = market_data.loc[:, market_data.columns != 'Market_Orientation']
#y_plain = market_data.loc[:, "Market_Orientation"]

In [64]:
#X and Y for market orientation: TESTING
#from sklearn.model_selection import train_test_split
#Xtrain, Xtest, ytrain, ytest = train_test_split(X_plain,y_plain,test_size = .3, random_state = 2021)

In [65]:
#X and Y for market orientation
from sklearn.model_selection import train_test_split

X = market_nums.loc[:, market_nums.columns != 'Market_Orientation']
y = market_nums.loc[:, "Market_Orientation"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size = .3, random_state = 2021)

In [68]:
#1. Fit OLS on the training dataset and compute the test error. Just numerics
reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

3.2186953150253945


In [69]:
#this is market orientation
betas_ppi = reg.coef_
max5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:20]]
min5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:10]]
betas_ppi[min5_index_ppi]
Xtrain.columns[min5_index_ppi]
pd.DataFrame([Xtrain.columns[min5_index_ppi], betas_ppi[min5_index_ppi]]).T

,0,1
0,value_livestock_production_USD_PPP_pHH_Yr,-2.21228e+13
1,value_livestock_prod_consumed_USD_PPP_pHH_Yr,2.01358e+13
2,Gender_MaleControl,1.56606e+13
3,Gender_FemaleControl,1.56606e+13
4,livestock_prodsales_USD_PPP_pHH_Yr,9.12637e+12
5,value_farm_produce_USD_PPP_pHH_Yr,9.06603e+11
6,crop_sales_USD_PPP_pHH_Yr,-6.91062e+11
7,total_income_USD_PPP_pHH_Yr,-2.06639e+11
8,offfarm_income_USD_PPP_pHH_Yr,1.32105e+11
9,farm_income_USD_PPP_pHH_Yr,-9.77692e+10


In [70]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:     Market_Orientation   R-squared (uncentered):                   0.143
Model:                            OLS   Adj. R-squared (uncentered):              0.140
Method:                 Least Squares   F-statistic:                              46.04
Date:                Wed, 09 Jun 2021   Prob (F-statistic):                   8.57e-201
Time:                        12:14:52   Log-Likelihood:                         -8935.0
No. Observations:                6645   AIC:                                  1.792e+04
Df Residuals:                    6621   BIC:                                  1.808e+04
Df Model:                          24                                                  
Covariance Type:            nonrobust                                                  
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------

In [71]:
#X and Y for market orientation ALL with dummy

Xtrain, Xtest, ytrain, ytest = train_test_split(X_market,y_market,test_size = .3, random_state = 50)
#1. Fit OLS on the training dataset and compute the test error. ALL with dummy

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

2.2913700844866826e+18


In [72]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:     Market_Orientation   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     25.05
Date:                Wed, 09 Jun 2021   Prob (F-statistic):               0.00
Time:                        12:14:53   Log-Likelihood:                -8052.7
No. Observations:                6645   AIC:                         1.638e+04
Df Residuals:                    6510   BIC:                         1.729e+04
Df Model:                         134                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [73]:
#this is market orientation
betas_ppi = reg.coef_
max5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:20]]
min5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:10]]
betas_ppi[min5_index_ppi]
Xtrain.columns[min5_index_ppi]
pd.DataFrame([Xtrain.columns[min5_index_ppi], betas_ppi[min5_index_ppi]]).T

,0,1
0,value_farm_produce_USD_PPP_pHH_Yr,-2.23571e+12
1,crop_sales_USD_PPP_pHH_Yr,1.58172e+12
2,value_livestock_production_USD_PPP_pHH_Yr,-8.68898e+11
3,value_livestock_prod_consumed_USD_PPP_pHH_Yr,8.30178e+11
4,value_crop_produce_USD_PPP_pHH_Yr,5.1116e+11
5,livestock_prodsales_USD_PPP_pHH_Yr,3.75075e+11
6,farm_income_USD_PPP_pHH_Yr,1.54668e+11
7,crop_name_1_geso,5.80788e+10
8,crop_name_1_spices,-4.43652e+10
9,crop_harvest_1_na,4.348e+10


In [74]:
#for PPI Threshold
market_PPI = fun.delete_id_columns(clean) #1
print("1")
market_PPI, pred_PPI = fun.drop_response_rows_with_NAs(market_PPI, "PPI_Likelihood", "Market_Orientation") #2
print("2")
market_PPI = fun.replace_NAN_with_na(market_PPI) #3
print("3")
market_PPI = fun.entry_to_lowercase(market_PPI) #4
print("4")
market_PPI = fun.remove_underscores_spaces(market_PPI) #5
print("5")
market_PPI = fun.convert_to_categorical(market_PPI) #6
print("6")
market_PPI = fun.impute_data(market_PPI)
print("7")
market_PPI.YEAR = market_PPI.YEAR.astype('category')
market_PPI = market_PPI.drop("Country", axis = 1)
market_PPI = standarize_data(market_PPI)

1
2
3
4
5
6
7


In [75]:
PPI_nums = market_PPI.select_dtypes(include='number')
X_PPI = get_dummyXs_y(market_PPI, "PPI_Likelihood")[0]
y_PPI = get_dummyXs_y(market_PPI, "PPI_Likelihood")[1]
print("8")

8


In [76]:
#X and Y for PPI_threshold
from sklearn.model_selection import train_test_split

X_p = PPI_nums.loc[:, PPI_nums.columns != 'PPI_Likelihood']
y_p = PPI_nums.loc[:, "PPI_Likelihood"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X_p,y_p,test_size = .3, random_state = 5)

In [77]:
#1. Fit OLS on the training dataset and compute the test error. Just numerics

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

0.9815192927524545


In [78]:
betas_ppi = reg.coef_
max5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:20]]
min5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:10]]
betas_ppi[min5_index_ppi]
Xtrain.columns[min5_index_ppi]
pd.DataFrame([Xtrain.columns[min5_index_ppi], betas_ppi[min5_index_ppi]]).T

,0,1
0,Food_Availability_kCal_MAE_day,0.830114
1,HHsizemembers,0.825778
2,TVA_USD_PPP_pmae_pday,-0.741225
3,HHsizeMAE,-0.716008
4,offfarm_income_USD_PPP_pHH_Yr,-0.150735
5,NrofMonthsWildFoodCons,0.1013
6,NrofMonthsFoodInsecure,0.0973118
7,score_HDDS_purchasedBadSeason,-0.068309
8,crop_count,0.0670102
9,total_income_USD_PPP_pHH_Yr,-0.0583924


In [79]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:         PPI_Likelihood   R-squared (uncentered):                   0.080
Model:                            OLS   Adj. R-squared (uncentered):              0.077
Method:                 Least Squares   F-statistic:                              26.54
Date:                Wed, 09 Jun 2021   Prob (F-statistic):                   3.46e-109
Time:                        12:14:59   Log-Likelihood:                         -9688.6
No. Observations:                7054   AIC:                                  1.942e+04
Df Residuals:                    7031   BIC:                                  1.958e+04
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------

In [80]:
#X and Y for market orientation WITH DUMMY
from sklearn.model_selection import train_test_split

X = market_nums.loc[:, market_nums.columns != 'Market_Orientation']
y = market_nums.loc[:, "Market_Orientation"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X_PPI,y_PPI,test_size = .3, random_state = 5)
#1. Fit OLS on the training dataset and compute the test error. ALL with dummy

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

6252668667763196.0


In [46]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:         PPI_Likelihood   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     30.58
Date:                Wed, 09 Jun 2021   Prob (F-statistic):               0.00
Time:                        11:33:29   Log-Likelihood:                -8524.0
No. Observations:                7054   AIC:                         1.728e+04
Df Residuals:                    6937   BIC:                         1.808e+04
Df Model:                         116                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [48]:
betas_ppi = reg.coef_
max5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:20]]
min5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:10]]
betas_ppi[min5_index_ppi]
Xtrain.columns[min5_index_ppi]
pd.DataFrame([Xtrain.columns[min5_index_ppi], betas_ppi[min5_index_ppi]]).T

,0,1
0,total_income_USD_PPP_pHH_Yr,-1.07545e+11
1,offfarm_income_USD_PPP_pHH_Yr,6.96692e+10
2,value_livestock_production_USD_PPP_pHH_Yr,6.36167e+10
3,crop_harvest_1_normalharvest,6.19289e+10
4,crop_harvest_1_na,6.19289e+10
5,crop_harvest_1_goodharvest,6.19289e+10
6,crop_harvest_1_badharvest,6.19289e+10
7,YEAR_2015,5.82716e+10
8,YEAR_2017,5.82716e+10
9,YEAR_2016,5.82716e+10
